In [23]:

import numpy as np
import glob 
import pandas as pd
import h5py # import to read hdf5
from pathlib import Path
import pyfstat
from scipy import stats
import os
from joblib import Parallel, delayed
import shutil
from tqdm import tqdm
import sys 


In [24]:
PREPROC_TRAIN_DATA = True

In [25]:


# root = "/Volumes/T7/gravitational-waves/kaggle-data"
root = "/media/viktor/T7/gravitational-waves-kaggle-2022"

def load_trained_files(train=True):
   if train:
      df = pd.read_csv(f'{root}/kaggle-data/train_labels.csv')
      df['filename'] = f'{root}/kaggle-data/train/' + df['id'].astype(str) + ".hdf5"
   else:
      test_files = glob.glob(f'{root}/kaggle-data/test/*.hdf5')
      df = pd.DataFrame({'filename': test_files})
      
   return df
print("[INFO] Loading files ...")
df = load_trained_files(train=PREPROC_TRAIN_DATA)
df


[INFO] Loading files ...


,id,target,filename
0,001121a05,1,/media/viktor/T7/gravitational-waves-kaggle-20...
1,004f23b2d,1,/media/viktor/T7/gravitational-waves-kaggle-20...
2,00a6db666,1,/media/viktor/T7/gravitational-waves-kaggle-20...
3,00f36a6ac,1,/media/viktor/T7/gravitational-waves-kaggle-20...
4,010a387db,1,/media/viktor/T7/gravitational-waves-kaggle-20...
...,...,...,...
598,fe38dbe64,1,/media/viktor/T7/gravitational-waves-kaggle-20...
599,feafd0d16,1,/media/viktor/T7/gravitational-waves-kaggle-20...
600,feeca844e,1,/media/viktor/T7/gravitational-waves-kaggle-20...
601,ff5ad023f,1,/media/viktor/T7/gravitational-waves-kaggle-20...


In [26]:
if PREPROC_TRAIN_DATA:
    df = df[df["target"] != -1].reset_index(drop=True)

In [27]:
df["name"] = df["filename"].apply(lambda x: x.split("/")[-1].split(".")[0])
df

,id,target,filename,name
0,001121a05,1,/media/viktor/T7/gravitational-waves-kaggle-20...,001121a05
1,004f23b2d,1,/media/viktor/T7/gravitational-waves-kaggle-20...,004f23b2d
2,00a6db666,1,/media/viktor/T7/gravitational-waves-kaggle-20...,00a6db666
3,00f36a6ac,1,/media/viktor/T7/gravitational-waves-kaggle-20...,00f36a6ac
4,010a387db,1,/media/viktor/T7/gravitational-waves-kaggle-20...,010a387db
...,...,...,...,...
595,fe38dbe64,1,/media/viktor/T7/gravitational-waves-kaggle-20...,fe38dbe64
596,feafd0d16,1,/media/viktor/T7/gravitational-waves-kaggle-20...,feafd0d16
597,feeca844e,1,/media/viktor/T7/gravitational-waves-kaggle-20...,feeca844e
598,ff5ad023f,1,/media/viktor/T7/gravitational-waves-kaggle-20...,ff5ad023f


In [28]:
if PREPROC_TRAIN_DATA:
    df["preprocessed_filename"] = df["name"].apply(lambda x: f"{root}/train/{x}.npy")
else:
    df["preprocessed_filename"] = df["name"].apply(lambda x: f"{root}/test/{x}.npy")

In [29]:
df.to_csv("train-preprocessed.csv" if PREPROC_TRAIN_DATA else "test-preprocessed.csv")
df

,id,target,filename,name,preprocessed_filename
0,001121a05,1,/media/viktor/T7/gravitational-waves-kaggle-20...,001121a05,/media/viktor/T7/gravitational-waves-kaggle-20...
1,004f23b2d,1,/media/viktor/T7/gravitational-waves-kaggle-20...,004f23b2d,/media/viktor/T7/gravitational-waves-kaggle-20...
2,00a6db666,1,/media/viktor/T7/gravitational-waves-kaggle-20...,00a6db666,/media/viktor/T7/gravitational-waves-kaggle-20...
3,00f36a6ac,1,/media/viktor/T7/gravitational-waves-kaggle-20...,00f36a6ac,/media/viktor/T7/gravitational-waves-kaggle-20...
4,010a387db,1,/media/viktor/T7/gravitational-waves-kaggle-20...,010a387db,/media/viktor/T7/gravitational-waves-kaggle-20...
...,...,...,...,...,...
595,fe38dbe64,1,/media/viktor/T7/gravitational-waves-kaggle-20...,fe38dbe64,/media/viktor/T7/gravitational-waves-kaggle-20...
596,feafd0d16,1,/media/viktor/T7/gravitational-waves-kaggle-20...,feafd0d16,/media/viktor/T7/gravitational-waves-kaggle-20...
597,feeca844e,1,/media/viktor/T7/gravitational-waves-kaggle-20...,feeca844e,/media/viktor/T7/gravitational-waves-kaggle-20...
598,ff5ad023f,1,/media/viktor/T7/gravitational-waves-kaggle-20...,ff5ad023f,/media/viktor/T7/gravitational-waves-kaggle-20...
